### **1. SSD video inference : practice**

In [ ]:
!pwd

/content


In [ ]:
!mkdir ./pretrained
!wget -O ./pretrained/ssd_inception_v2_coco_2017_11_17.tar.gz http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2017_11_17.tar.gz
!wget -O ./pretrained/ssd_config_01.pbtxt https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/ssd_inception_v2_coco_2017_11_17.pbtxt

--2022-07-23 04:06:44--  http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2017_11_17.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.148.128, 2607:f8b0:4001:c54::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.148.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278126337 (265M) [application/x-tar]
Saving to: ‘./pretrained/ssd_inception_v2_coco_2017_11_17.tar.gz’

./pretrained/ssd_in 100%[===================>] 265.24M   136MB/s    in 2.0s    

2022-07-23 04:06:46 (136 MB/s) - ‘./pretrained/ssd_inception_v2_coco_2017_11_17.tar.gz’ saved [278126337/278126337]

--2022-07-23 04:06:46--  https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/ssd_inception_v2_coco_2017_11_17.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent

In [ ]:
!tar -xvf ./pretrained/ssd_inception_v2_coco_2017_11_17.tar.gz -C ./pretrained

ssd_inception_v2_coco_2017_11_17/
ssd_inception_v2_coco_2017_11_17/model.ckpt.index
ssd_inception_v2_coco_2017_11_17/model.ckpt.meta
ssd_inception_v2_coco_2017_11_17/frozen_inference_graph.pb
ssd_inception_v2_coco_2017_11_17/model.ckpt.data-00000-of-00001
ssd_inception_v2_coco_2017_11_17/saved_model/
ssd_inception_v2_coco_2017_11_17/saved_model/saved_model.pb
ssd_inception_v2_coco_2017_11_17/saved_model/variables/
ssd_inception_v2_coco_2017_11_17/checkpoint


In [ ]:
import cv2

cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/ssd_inception_v2_coco_2017_11_17/frozen_inference_graph.pb', 
                                       './pretrained/ssd_config_01.pbtxt')

In [ ]:
labels_to_names = {1:'person',2:'bicycle',3:'car',4:'motorcycle',5:'airplane',6:'bus',7:'train',8:'truck',9:'boat',10:'traffic light',
                    11:'fire hydrant',12:'street sign',13:'stop sign',14:'parking meter',15:'bench',16:'bird',17:'cat',18:'dog',19:'horse',20:'sheep',
                    21:'cow',22:'elephant',23:'bear',24:'zebra',25:'giraffe',26:'hat',27:'backpack',28:'umbrella',29:'shoe',30:'eye glasses',
                    31:'handbag',32:'tie',33:'suitcase',34:'frisbee',35:'skis',36:'snowboard',37:'sports ball',38:'kite',39:'baseball bat',40:'baseball glove',
                    41:'skateboard',42:'surfboard',43:'tennis racket',44:'bottle',45:'plate',46:'wine glass',47:'cup',48:'fork',49:'knife',50:'spoon',
                    51:'bowl',52:'banana',53:'apple',54:'sandwich',55:'orange',56:'broccoli',57:'carrot',58:'hot dog',59:'pizza',60:'donut',
                    61:'cake',62:'chair',63:'couch',64:'potted plant',65:'bed',66:'mirror',67:'dining table',68:'window',69:'desk',70:'toilet',
                    71:'door',72:'tv',73:'laptop',74:'mouse',75:'remote',76:'keyboard',77:'cell phone',78:'microwave',79:'oven',80:'toaster',
                    81:'sink',82:'refrigerator',83:'blender',84:'book',85:'clock',86:'vase',87:'scissors',88:'teddy bear',89:'hair drier',90:'toothbrush',
                    91:'hair brush'}

In [ ]:
video_input_path = ('./John_Wick_small.mp4')
video_output_path = ('./drive/MyDrive/Colab Notebooks/update/dl_cv/4_dl_cv_ssd_inference/data/John_Wick_small_output_01.mp4')

cap = cv2.VideoCapture(video_input_path)
codec = cv2.VideoWriter_fourcc(*'XVID')
video_fps = round(cap.get(cv2.CAP_PROP_FPS))
video_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_writer = cv2.VideoWriter(video_output_path, codec, video_fps, video_size)
print(f'frame count : {frame_count}, video fps : {video_fps}, video size : {video_size}')

frame count : 58, video fps : 30, video size : (1072, 460)


In [ ]:
import time

rgb_green = (0, 255, 0)
rgb_red = (0, 0, 255)

while True:
  hasframe, frame = cap.read()
  if not hasframe:
    print('process completed')
    break
  
  frame_height = frame.shape[0]
  frame_width = frame.shape[1]

  cv_net.setInput(cv2.dnn.blobFromImage(frame, swapRB=True, crop=False))
  start = time.time()
  cv_out = cv_net.forward()
  
  for detection in cv_out[0, 0, :, :]:
    class_id = int(detection[1])
    class_confidence = detection[2]
    if class_confidence > 0.5:
      left = detection[3] * frame_width
      top = detection[4] * frame_height
      right = detection[5] * frame_width
      bottom = detection[6] * frame_height 
      caption = '{} : {:.4f}'.format(labels_to_names[class_id], class_confidence)
      cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), color=rgb_green, thickness=2)
      cv2.putText(frame, caption, (int(left), int(top-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color=rgb_red, thickness=1)
  print(f'detection time : {round(time.time()-start, 2)}초')
  video_writer.write(frame)

video_writer.release()
cap.release()

detection time : 1.28초
detection time : 1.15초
detection time : 1.14초
detection time : 1.15초
detection time : 1.19초
detection time : 1.14초
detection time : 1.15초
detection time : 1.15초
detection time : 1.13초
detection time : 1.12초
detection time : 1.13초
detection time : 1.14초
detection time : 1.13초
detection time : 1.15초
detection time : 1.13초
detection time : 1.18초
detection time : 1.21초
detection time : 1.13초
detection time : 1.13초
detection time : 1.14초
detection time : 1.15초
detection time : 1.16초
detection time : 1.13초
detection time : 1.13초
detection time : 1.14초
detection time : 1.13초
detection time : 1.13초
detection time : 1.13초
detection time : 1.13초
detection time : 1.14초
detection time : 1.15초
detection time : 1.13초
detection time : 1.13초
detection time : 1.14초
detection time : 1.13초
detection time : 1.13초
detection time : 1.14초
detection time : 1.13초
detection time : 1.13초
detection time : 1.15초
detection time : 1.13초
detection time : 1.13초
detection time : 1.13초
detection t

### **2. SSD video inference : functionalize the code**

In [ ]:
def get_detected_video(cv_net, video_input_path, video_output_path, threshold=0.5, print_time=True):
  cap = cv2.VideoCapture(video_input_path)
  codec = cv2.VideoWriter_fourcc(*'XVID')
  video_fps = round(cap.get(cv2.CAP_PROP_FPS))
  video_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  
  video_writer = cv2.VideoWriter(video_output_path, codec, video_fps, video_size)
  print(f'frame count : {frame_count}, video fps : {video_fps}, video size : {video_size}')

  rgb_green = (0, 255, 0)
  rgb_red = (0, 0, 255)

  while True:
    hasframe, frame = cap.read()
    if not hasframe:
      print('process completed')
      break
    
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    cv_net.setInput(cv2.dnn.blobFromImage(frame, swapRB=True, crop=False))
    start = time.time()
    cv_out = cv_net.forward()

    for detection in cv_out[0, 0, :, :]:
      class_id = int(detection[1])
      class_confidence = detection[2]
      if class_confidence > threshold:
        left = detection[3] * frame_width
        top = detection[4] * frame_height
        right = detection[5] * frame_width
        bottom = detection[6] * frame_height
        caption = '{} : {:.4f}'.format(labels_to_names[class_id], class_confidence)
        cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), color=rgb_green, thickness=2)
        cv2.putText(frame, caption, (int(left), int(top-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color=rgb_red, thickness=1)
    
    if print_time:
      print(f'detection time : {round(time.time()-start, 2)}초')
    else:
      pass
    video_writer.write(frame)
  
  video_writer.release()
  cap.release()

In [ ]:
video_input_path = ('./John_Wick_small.mp4')
video_output_path = ('./drive/MyDrive/Colab Notebooks/update/dl_cv/4_dl_cv_ssd_inference/data/John_Wick_small_output_02.mp4')

get_detected_video(cv_net, video_input_path, video_output_path, threshold=0.5, print_time=True)

frame count : 58, video fps : 30, video size : (1072, 460)
detection time : 1.17초
detection time : 1.15초
detection time : 1.13초
detection time : 1.15초
detection time : 1.15초
detection time : 1.13초
detection time : 1.12초
detection time : 1.15초
detection time : 1.13초
detection time : 1.12초
detection time : 1.13초
detection time : 1.13초
detection time : 1.13초
detection time : 1.13초
detection time : 1.13초
detection time : 1.13초
detection time : 1.15초
detection time : 1.15초
detection time : 1.12초
detection time : 1.13초
detection time : 1.14초
detection time : 1.12초
detection time : 1.15초
detection time : 1.13초
detection time : 1.12초
detection time : 1.16초
detection time : 1.13초
detection time : 1.14초
detection time : 1.13초
detection time : 1.13초
detection time : 1.13초
detection time : 1.15초
detection time : 1.13초
detection time : 1.12초
detection time : 1.15초
detection time : 1.12초
detection time : 1.54초
detection time : 2.06초
detection time : 1.85초
detection time : 1.12초
detection time : 1.14